In [1]:
%%capture
!pip install datasets
!pip install ollama
!pip install llama-index-llms-ollama

!pip install llama-index


In [2]:
! sudo apt-get install -y pciutils
! curl https://ollama.ai/install.sh | sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package pciutils
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13344    0 13344    0     0  24711      0 --:--:-- --:--:-- --:--:-- 24665
>>> Downloading Linux amd64 CLI
######################################################################## 100.0%##O#- #                                                                       
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

In [4]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Couldn't find '/home/codespace/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIESEaB/hzXkACtM54zLEpFiLfouZNsWGjjMrVr3b1JNb



2024/08/24 12:38:09 routes.go:1125: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/codespace/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-08-24T12:38:09.920Z level=INFO source=images.go:782 msg="total blobs: 0"
time=2024-08-24T12:38:09.920Z level=INFO source=images.go:790 msg="total unused blobs removed: 0"
time=2024-08-24

In [20]:
! ollama run mistral

[GIN] 2024/08/24 - 12:58:42 | 200 |      21.941µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/08/24 - 12:58:42 | 404 |      88.084µs |       127.0.0.1 | POST     "/api/show"
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  

time=2024-08-24T12:58:45.030Z level=INFO source=download.go:175 msg="downloading ff82381e2bea in 42 100 MB part(s)"


pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   1% ▕                ▏  26 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕                ▏  69 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕                ▏  74 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕                ▏ 140 MB/4.1 GB  105 MB/s     37spulling manifest 
pulling ff82

time=2024-08-24T12:59:04.450Z level=INFO source=download.go:175 msg="downloading 43070e2d4e53 in 1 11 KB part(s)"


pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53...   0% ▕                ▏    0 B/ 11 KB                  pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53...   0% ▕                ▏    0 B/ 11 KB                  pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53...   0% ▕                ▏    0 B/ 11 KB                  pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53...   0% ▕                ▏    0 B/ 11 KB                  pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4

...

⠼ ⠴ ⠦ ⠧ ⠇ ⠏ 

llama_kv_cache_init:        CPU KV buffer size =  1024.00 MiB
llama_new_context_with_model: KV self size  = 1024.00 MiB, K (f16):  512.00 MiB, V (f16):  512.00 MiB
llama_new_context_with_model:        CPU  output buffer size =     0.56 MiB
llama_new_context_with_model:        CPU compute buffer size =   560.01 MiB
llama_new_context_with_model: graph nodes  = 1030
llama_new_context_with_model: graph splits = 1


⠋ ⠙ INFO [main] model loaded | tid="137931848083328" timestamp=1724504360
⠹ [GIN] 2024/08/24 - 12:59:20 | 200 |  3.275480694s |       127.0.0.1 | POST     "/api/chat"
>>> Send a message (/? for help)

time=2024-08-24T12:59:20.460Z level=INFO source=server.go:632 msg="llama runner started in 3.26 seconds"



Use Ctrl + d or /bye to exit.
>>> Send a message (/? for help)
>>> Send a message (/? for help)

In [21]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Error: listen tcp 0.0.0.0:11434: bind: address already in use


In [7]:
from datasets import load_dataset

ds = load_dataset("tmnam20/Vietnamese-News", "dantri")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 100396/100396 [00:03<00:00, 30165.28 examples/s]


In [8]:
len(ds["train"]["description"])

100396

In [9]:
# prompt: sort the description contain number like "1" , "5.000" , "6,6"

import re

def has_number(text):
  return bool(re.search(r'\d', text))

# Filter descriptions containing numbers
descriptions_with_numbers = [desc for desc in ds["train"]["description"] if has_number(desc)]

# Sort the filtered descriptions
sorted_descriptions = sorted(descriptions_with_numbers)

# Print the sorted descriptions



In [10]:
len(sorted_descriptions)

51571

In [11]:
import pandas as pd

In [12]:
sorted_df = pd.DataFrame(sorted_descriptions)
sorted_df.columns = ["original"]

In [13]:
sorted_df.head()

,original
0,Hơn 100 hecta rừng phong đang dần thay lá đỏ ...
1,Hotgirl Vi Á - người được mệnh danh là nữ hoà...
2,Sự việc diễn ra vào ngày 1/5 trên tuyến đường...
3,"""'Nhân duyên nếu có một lần được gặp gỡ, thì n..."
4,"""1,2,3 dô! Lên bia..."" là những âm thanh chúng..."


In [22]:
def give_prompt(p):
    prompt = f"""
Chuyển đổi tất cả các con số trong đoạn văn dưới đây sang dạng chữ nói bằng tiếng Việt, bao gồm số đếm, giá trị tiền tệ, và ngày tháng. Đầu ra cần tuân theo cấu trúc sau:

**Dữ liệu đầu vào:**
{{input_text}}

**Dữ liệu đầu ra:**
{{output_text}}

Ví dụ:
- Dữ liệu đầu vào: U23
- Dữ liệu đầu ra: u hai ba

- Dữ liệu đầu vào: 76
- Dữ liệu đầu ra: bảy sáu

- Dữ liệu đầu vào: 5.000
- Dữ liệu đầu ra: năm nghìn

- Dữ liệu đầu vào: 5.000.000
- Dữ liệu đầu ra: năm triệu

- Dữ liệu đầu vào: Ngày 12/11
- Dữ liệu đầu ra: Ngày mười hai tháng mười một

- Dữ liệu đầu vào: 1 tỷ USD
- Dữ liệu đầu ra: một tỷ đô la

- Dữ liệu đầu vào: 500.000 USD
- Dữ liệu đầu ra: năm trăm nghìn đô la

- Dữ liệu đầu vào: năm 2020
- Dữ liệu đầu ra: năm hai không hai mươi

- Dữ liệu đầu vào: 123456 là
- Dữ liệu đầu ra: một hai ba bốn năm sáu là

Dữ liệu đầu vào: {p}

Hãy trả về dữ liệu đầu ra dưới định dạng nêu trên, không có chữ số đặc biệt.
"""
    return prompt

In [15]:
import re

def process_output(text):
    # Xử lý ký tự không mong muốn như \x1d
    clean_text = text.replace('\x1d', '')
    
    # Tìm kiếm và trích xuất phần **Dữ liệu đầu ra:**
    match = re.search(r'\*\*Dữ liệu đầu ra:\*\*(.*)', clean_text, re.DOTALL)
    
    if match:
        # Trả về phần "Dữ liệu đầu ra"
        return match.group(1).strip()
    else:
        # Nếu không tìm thấy, trả về None hoặc thông báo lỗi
        return None

In [16]:
test_df   = sorted_df["original"].sample(10)

In [17]:
test_list = test_df.tolist()

In [23]:
import time
url = 'http://localhost:11434/api/chat'
response_list = []
start_time = time.time()
for p in test_list:

    payload = {
        "model": "mistral",
        "temperature": 0.1,
        "stream": False,
        "messages": [
            {"role": "system", "content": "You are AI assitant and can read the news by spoken language"},
            {"role": "user", "content": give_prompt(p)}
        ]
    }

    response = requests.post(url, json=payload)
    message_str = response.content.decode('utf-8')
    message_dict = json.loads(message_str)
    response_list.append(process_output(message_dict['message']['content']))

end_time = time.time()

# Calculate the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time:.2f} seconds")


[GIN] 2024/08/24 - 13:05:37 | 200 |          3m8s |             ::1 | POST     "/api/chat"
[GIN] 2024/08/24 - 13:07:27 | 200 |         1m50s |             ::1 | POST     "/api/chat"
[GIN] 2024/08/24 - 13:07:42 | 500 |  14.11496026s |             ::1 | POST     "/api/chat"


KeyboardInterrupt: 

In [96]:
result = dict(zip(test_list, response_list))
result_df = pd.DataFrame(result.items(), columns=['original', 'response'])


In [19]:
response_list

['Chuyến bay kéo dài hai mươi lăm phút, du khách được chiêm ngưỡng khung cảnh tuyệt đẹp của vịnh Hạ Long từ trên cao, lượn ngang qua những dãy núi có hình thù độc đáo như mọc lên từ mặt nước.',
 '4 năm qua, vườn chà là của ông Tuyến là điểm đến ấn tượng, thu hút du khách khi ghé thăm làng hoa Sa Đéc (Đồng Tháp). Nhiều người sẵn sàng vượt hàng trăm km đến đây chỉ để chiêm ngưỡng cây chà là.',
 'Truyền thông Ukraine đưa tin lực lượng Kiev đã bắn hạ một máy bay quân sự A-50 của Nga trên biển Azov hôm mười bốn tháng một.',
 'Nhiều cổ động viên (CĐV) Thái Lan đã lên tiếng cho rằng U23 Việt Nam đang cố tình tránh U23 Thái Lan ở vòng bán kết Tứ hùng Đông Nam Á Mười một.',
 'Chiều hai mươi lăm tháng một, trao đổi với PV Dân trí, ông Huỳnh Vĩnh Khang - Chánh Văn phòng Sở GD-ĐT tỉnh Khánh Hòa, cho biết đã có kết quả vụ xác minh việc lộ đề kiểm tra học kỳ 1 lớp twelve tại tỉnh này. Theo đó, một giáo viên đã “tuồn” đề Toán trước cho một học sinh mà mình dạy kèm.',
 'Sau khi mua vé buffet, nhóm bốn

In [79]:
result_df["response"]



0    Hơn một trăm hecta rừng phong đang dần thay lá...
1    Thiên tài nữ hoàng livestream Hotgirl Vi Á vừa...
2    Sự việc diễn ra vào ngày một tháng năm trên tu...
3    Nhân duyên, nếu có một lần được gặp gỡ, thì ng...
4                          Một, hai, ba dô! Lên bia...
5                                mười lời thề danh dự.
6    Mười giờ mới mở cửa, khách hàng đã xếp hàng dà...
7    mười hai nghìn ba trăm bốn mươi năm sáu đứng đ...
Name: response, dtype: object

In [34]:
result.keys()

dict_keys(['\x08Hơn 100 hecta rừng phong đang dần thay lá đỏ tạo nên một cảnh đẹp vô cùng kỳ thú, khác lạ tại núi Tam Ban (Chí Linh, Hải Dương).', '\x1dHotgirl Vi Á - người được mệnh danh là nữ hoàng livestream trong hệ sinh thái mua sắm trực tuyến của Trung Quốc vừa lọt danh sách 500 người giàu nhất Trung Quốc năm 2021.', '\x1dSự việc diễn ra vào ngày 1/5 trên tuyến đường Lê Hiến Tông, huyện Thọ Xuân, tỉnh Thanh Hóa.', '"\'Nhân duyên nếu có một lần được gặp gỡ, thì ngay từ ánh mắt đầu tiên, anh đã biết định mệnh chính là em", đó là cảm xúc ngọt ngào mà Vũ Hồ Bắc dành tặng bạn gái Thúy An vào dịp kỉ niệm 1 năm yêu.', '"1,2,3 dô! Lên bia..." là những âm thanh chúng tôi ghi nhận được ở nhiều quán nhậu tại TPHCM. Những "ma men" sau khi sử dụng thức uống có cồn vẫn điều khiển xe máy một mình và không đội mũ bảo hiểm.', '"10 lời thề danh dự" thể hiện tinh thần và khí chất cách mạng của Quân đội nhân dân Việt Nam, là niềm vinh dự tự hào cho các học viên Học viện Kỹ thuật Quân sự khi tuyên th

In [51]:
test_df.head(10)

0    Hơn 100 hecta rừng phong đang dần thay lá đỏ ...
1    
Hotgirl Vi Á - người được mệnh danh là nữ hoà...
2    
Sự việc diễn ra vào ngày 1/5 trên tuyến đường...
3    "'Nhân duyên nếu có một lần được gặp gỡ, thì n...
4    "1,2,3 dô! Lên bia..." là những âm thanh chúng...
5    "10 lời thề danh dự" thể hiện tinh thần và khí...
6    "10 lời thề danh dự" thể hiện tinh thần và khí...
7    "10h mới mở cửa mà 9h khách hàng đã xếp hàng d...
8    "123456" đứng đầu trong danh sách những mật kh...
9    "123456" đứng đầu trong danh sách những mật kh...
Name: original, dtype: object

In [47]:
test_df

0           Hơn 100 hecta rừng phong đang dần thay lá đỏ ...
1           
Hotgirl Vi Á - người được mệnh danh là nữ hoà...
2           
Sự việc diễn ra vào ngày 1/5 trên tuyến đường...
3           "'Nhân duyên nếu có một lần được gặp gỡ, thì n...
4           "1,2,3 dô! Lên bia..." là những âm thanh chúng...
5           "10 lời thề danh dự" thể hiện tinh thần và khí...
6           "10 lời thề danh dự" thể hiện tinh thần và khí...
7           "10h mới mở cửa mà 9h khách hàng đã xếp hàng d...
8           "123456" đứng đầu trong danh sách những mật kh...
9           "123456" đứng đầu trong danh sách những mật kh...
response    [Hơn 100 hecta rừng phong đang dần thay lá đỏ ...
Name: original, dtype: object

In [24]:
prompt

'Chuyển đổi các con số từ dạng chữ viết sang dạng chữ nói bằng tiếng Việt. Các con số đưa ra dưới đây có thể là số đếm, giá trị tiền tệ hoặc ngày tháng. Làm theo các hướng dẫn này:\n\nChuyển đổi số đếm:\n\n100 thành “một trăm”\n15.000 thành “mười năm nghìn”\nChuyển đổi số tiền tệ:\n\n13 tỷ USD thành “mười ba tỷ đô la”\n500.000 VNĐ thành “năm trăm nghìn đồng”\nChuyển đổi ngày tháng:\n\nNgày 15/8 thành “ngày mười năm tháng tám”\nNgày 1/1 thành “ngày một tháng một”\nDữ liệu đầu vào sẽ được đưa ra trong đoạn văn sau:\n"ngày 15/7, giá vàng chạm mốc 83.2 triệu đồng"\n\n\nHãy chuyển đổi toàn bộ các con số trong đoạn văn trên thành dạng chữ nói bằng tiếng Việt theo các hướng dẫn trên.'